In [1]:
import numpy as np
import matplotlib.pyplot as plt

import atm_adapter

import tqdm
import glob, os

from astropy.io import fits

%matplotlib inline

# Prepare models for publication

## HRFIT atmospheres

Use the cell below to convert ATLAS and PHOENIX model atmospheres calculated with HRFIT into FITS files.

**Note:** Choose between `adapt_phoenix()` and `adapt_atlas()`

In [8]:
output_dir = '/media/roman/TeraTwo/website/ca2/'
models = glob.glob('/media/roman/TeraTwo/christian/ca2/PHOENIX/*')

for model in tqdm.tqdm_notebook(models):
    try:
        atm_adapter.adapt_phoenix(model, output_dir + model.split('/')[-1] + '.fits')
    except:
        print('Cannot pack {}'.format(model))

/tmp/ipykernel_230717/56918304.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for model in tqdm.tqdm_notebook(models):


  0%|          | 0/404 [00:00<?, ?it/s]

## [LEGACY] ATMOS.UCSD.EDU atmospheres

Use the cell below to convert the models on atmos.ucsd.edu into FITS files.

In [ ]:
import json, hashlib, gzip

models = glob.glob('/media/roman/TeraTwo/website/atlas/*')
output = '/media/roman/TeraTwo/website/output/'              # Save output here

# MD5 hashes and identifiers of populations present in the input directory
pops = {
    '81f526240b1118a8371a27ccffdfb7d7': 'nominal',
    'b71187e90bb2b2930ceab9350ef1879c': 'HMHA',
    '6a1a292df6872327383cd557d6fdfda2': 'nominal',
    '01f9fb93785d5630b841b6b901fd3a9b': 'HMMA',
    'e4490f13948d6d242f623c977be5ce59': 'popIII',
    '06ddffd8c02366baadffa02879a00282': 'HMET',
    'dd5a0ce7edc86228c0208ff442e2ebd2': 'HMHA',
    'b263e442553d6b3dbdf04733d7021e5b': 'HMMO',
    'f54e05ce851c74e2d90439e25e8d54d4': 'LMHO',
    '4fea898c6876647164e07637419a9d8b': 'HMMA',
    'd3e776168b85c3919357d59f17b6910b': 'HMET',
    '9641ed477799df2e9888962429321d2e': 'HMMO',
    'e507badb75ffb9e60ee71eaa3139bb53': 'LMHO'
}

def meta_get_value(meta, field):
    for category in meta:
        for item in category['category_content']:
            if item == field:
                return category['category_content'][item]['value']
    return False

for model in tqdm.tqdm_notebook(models):
    # Load and test model
    try:
        spectrum = np.loadtxt(model + '/spect.dat.gz', delimiter = ',', unpack = True)
        f = gzip.open(model + '/spect.dat.gz', 'r')
        spectrum_header = ''
        while spectrum_header.find(',') == -1:
            spectrum_header = f.readline().decode('utf-8').replace('#', '')
        f.close()
        spectrum_header = list(map(lambda s: s.strip(), spectrum_header.split(',')))
        spectrum = dict(zip(spectrum_header, spectrum))
        structure = np.loadtxt(model + '/struc.dat.gz', delimiter = ',', unpack = True)
        f = gzip.open(model + '/struc.dat.gz', 'r')
        structure_header = ''
        while structure_header.find(',') == -1:
            structure_header = f.readline().decode('utf-8').replace('#', '')
        f.close()
        structure_header = list(map(lambda s: s.strip(), structure_header.split(',')))
        structure = dict(zip(structure_header, structure))
        f = open(model + '/meta.json', 'r')
        meta = json.loads(f.read())
        f.close()
    except:
        print('{} is corrupted!'.format(model))
        continue

    # Extract abundances and hence determine the output directory
    found = False
    for i in range(len(meta)):
        if meta[i]['category_title'] == 'Elemental composition':
            if ('alpha' in meta[i]['category_content']) and (float(meta[i]['category_content']['alpha']['value']) != 0.0):
                print('Non-zero alpha in {}'.format(model))
            abundances = meta[i]['category_content']['zscale']['value'] + '|' + '|' + meta[i]['category_content']['eheu']['value']
            found = True
            break
    if not found:
        print('Cannot find abundances for {}'.format(model))
        continue
    if (hash := hashlib.md5(abundances.encode('utf-8')).hexdigest()) not in pops:
        raise ValueError('Found undefined population: {}'.format(hash))
    output_dir = pops[hashlib.md5(abundances.encode('utf-8')).hexdigest()]
    output_dir = output + output_dir + '/' + output_dir

    # Build FITS headers
    headers = {
        'SOFTWARE': meta_get_value(meta, 'software'),
        'TEFF': float(meta_get_value(meta, 'teff')),
        'LOGG': float(meta_get_value(meta, 'logg')),
        'ZSCALE': float(meta_get_value(meta, 'zscale')),
        'RADIUS': float(meta_get_value(meta, 'r0')),
        'X': float(meta_get_value(meta, 'X')),
        'Y': float(meta_get_value(meta, 'Y')),
        'Z': float(meta_get_value(meta, 'Z')),
        'VTURB': float(meta_get_value(meta, ['synthe_vturb', 'xi'][meta_get_value(meta, 'software').lower().find('phoenix') != -1]).replace(' + convection', '')),
        'MIXLEN': float(meta_get_value(meta, 'mixlen')),
        'DUST': meta_get_value(meta, 'igrains') == 'Enabled',
        'CLOUDS': meta_get_value(meta, 'use_clouds') == 'Enabled',
        'CLDALPHA': float(meta_get_value(meta, 'cloud_covering_factor')),
        'SETTL': meta_get_value(meta, 'settleos') == 'Enabled',
        'GEOMETRY': ['PLANE-PARALLEL', 'SPHERICAL'][meta_get_value(meta, 'software').lower().find('phoenix') != -1],
        'AIRORVAC': ([meta_get_value(meta, 'synthe_mode'), 'VAC'][meta_get_value(meta, 'software').lower().find('phoenix') != -1]).upper(),
    }

    # Software-specific corrections
    if meta_get_value(meta, 'software').lower().find('phoenix') != -1:
        spectrum['Intensity [erg s^-1 cm^-2 A^-1 sr^-1]'] = spectrum['Intensity [erg s^-1 cm^-2 A^-1 sr^-1]'] / np.pi ** 2.0 * 10
        tau_label = 'Optical depth at {} AA'.format(meta_get_value(meta, 'wltau'))
    else:
        structure['Convective flux fraction (Fconv)'] = structure['Convective flux fraction (Fconv)'] / (atm_adapter.scp.sigma * 1e3 * headers['TEFF'] ** 4 / (4 * np.pi))
        tau_label = 'Rosseland optical depth'


    # Save to FITS
    tables = []

    columns = {
        'wl': fits.Column(name = 'Wavelength', array = spectrum['Wavelength [A]'], format = 'D', unit = 'AA'),
        'flux': fits.Column(name = 'Flux density', array = spectrum['Intensity [erg s^-1 cm^-2 A^-1 sr^-1]'] * np.pi, format = 'D', unit = 'ERG/S/CM2/AA'),
    }
    hdr = fits.Header()
    hdr['TABLE'] = 'Emergent spectrum'
    tables += [fits.BinTableHDU.from_columns(columns.values(), header = hdr)]

    columns = {
        'tau': fits.Column(name = tau_label, array = structure['Optical depth (tau)'], format = 'D', unit = 'NONE'),
        'temp': fits.Column(name = 'Temperature', array = structure['Temperature [K]'], format = 'D', unit = 'K'),
        'pgas': fits.Column(name = 'Gas pressure', array = structure['Gas pressure [bar]'] * 1e6, format = 'D', unit = 'DYN/CM2'),
        'cmass': fits.Column(name = 'Column mass density', array = structure['Mass column density [g cm^-2]'], format = 'D', unit = 'G/CM2'),
        'fconv': fits.Column(name = 'Convective flux fraction', array = structure['Convective flux fraction (Fconv)'], format = 'D', unit = 'NONE'),
    }
    hdr = fits.Header()
    hdr['TABLE'] = 'Atmospheric structure'
    tables += [fits.BinTableHDU.from_columns(columns.values(), header = hdr)]

    eheu = meta_get_value(meta, 'eheu').split('|')
    element_name = []
    element_abundance = []
    for i in range(3, 100):
        if i not in atm_adapter.chemical_symbols:
            continue
        for element in eheu:
            element = element.split(':')
            if element[0] == atm_adapter.chemical_symbols[i]:
                element_name += [element[0]]
                element_abundance += [float(element[1])]
    columns = {
        'element': fits.Column(name = 'Element', array = element_name, format = 'A2'),
        'eheu': fits.Column(name = 'Relative abundance', array = element_abundance, format = 'D', unit = 'DEX'),
    }
    hdr = fits.Header()
    hdr['TABLE'] = 'Chemical composition'
    tables += [fits.BinTableHDU.from_columns(columns.values(), header = hdr)]

    hdr = fits.Header()
    for key in headers:
        hdr[key] = headers[key]

    hdul = fits.HDUList([fits.PrimaryHDU(header=hdr)] + tables)
    hdul.writeto(output_dir + '_{}_{}.fits'.format(int(headers['TEFF']), headers['LOGG']), overwrite = False)

## HRFIT interiors

Use the cell below to prepare MESA tables for export.

In [32]:
import shutil


# Export the individual MESA models
def export_MESA_model(model, export_to):
    files = ['history_columns.list', 'inlist', 'inlist_pgstar', 'inlist_project', 'LOGS/history.data', 'output.mod', 'output.out', 'src/*']
    for file in files:
        for file in glob.glob(model + '/' + file):
            destination = export_to + '/' + os.path.relpath(file, model + '/..')
            os.makedirs(os.path.dirname(destination), exist_ok = True)
            shutil.copy(file, destination)

# Destination
basedir = '/media/roman/TeraTwo/website/GAv0/ca2/MESA/'

# Population name
population = 'ca2'

# Population directory
pop_dir = '/media/roman/TeraTwo/christian/{}'.format(population)

# Input and output directories (output with respect to basedir)
models = {
    '{}/MESA/photosphere/'.format(pop_dir): 'models/photosphere/',
    '{}/MESA/tau_100/'.format(pop_dir): 'models/tau_100/',
}

for model_dir in models:
    for model in glob.glob(model_dir + '/*'):
        if not os.path.isfile(model + '/inlist_project'):
            continue
        export_MESA_model(model, basedir + '/' + models[model_dir])

# Export the BC tables
shutil.copytree('{}/TABLES/MESA_BC/'.format(pop_dir), destination := (basedir + '/tables'))
os.remove(destination + '/diagnostic.pkl')

# Export the MESA patch
shutil.copy('/home/roman/MESA/custom.patch', basedir)

'/media/roman/TeraTwo/website/GAv0/ca2/MESA/custom.patch'

## Test FITS files

Load headers from all FITS files in a directory for simultaneous inspection.

In [35]:
models = glob.glob('/media/roman/TeraTwo/website/GAv0/ca1/PHOENIX/*')

result = {}
eheu = {}
abs_eheu = {}

for model in tqdm.tqdm_notebook(models):
    h = fits.open(model)

    # Get main headers
    header = h[0].header
    for key in header:
        if key in result:
            result[key] += [header[key]]
        else:
            result[key] = [header[key]]

    # Locate abundances hdul
    index = 0
    for i in range(1, len(h)):
        if ('TABLE' in h[i].header) and (h[i].header['TABLE'] == 'Chemical composition'):
            index = i
    if index < 0:
        raise ValueError('Cannot find abundances')

    # Load abundances
    for element in h[index].data:
        if len(element) > 1:
            if element[0] in eheu:
                eheu[element[0]] += [element[1]]
            else:
                eheu[element[0]] = [element[1]]
        if len(element) > 2:
            if element[0] in abs_eheu:
                abs_eheu[element[0]] += [element[2]]
            else:
                abs_eheu[element[0]] = [element[2]]
    h.close()

# Uniquify all output
for key in result:
    result[key] = np.unique(result[key])

for key in eheu:
    eheu[key] = np.unique(eheu[key])

for key in abs_eheu:
    abs_eheu[key] = np.unique(abs_eheu[key])

/tmp/ipykernel_258797/3954192025.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for model in tqdm.tqdm_notebook(models):


  0%|          | 0/358 [00:00<?, ?it/s]